In [ ]:
!conda install -y gdown

# Loading Packages and Weights for My Custom model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import os 

import torchvision
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch.nn.functional as F

import PIL

from sklearn.metrics import confusion_matrix

from collections import OrderedDict

import time
import gc

gc.enable()
torch.__version__

In [ ]:
!gdown --id 1fpu2IATXJBoCD0adJDVjGYZ0qwDnQ4e8

In [ ]:
os.path.exists("/kaggle/working/vgg_cat_vs_dog.h5")

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)

# Extracting the training and test set

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/" + "train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/" + "test.zip","r") as z:
    z.extractall(".")

In [ ]:
def loadFileNames(trainTestFlag=True, start=0, end=12500, root=os.getcwd()):
    fileNames = []
    for ind in range(start, end):
        fileName = []
        if trainTestFlag:
            fileName = ["cat." + str(ind) + ".jpg", "dog." + str(ind) + ".jpg"]
        else:
            fileName = [str(ind) + ".jpg"]
        
        for ind, p in enumerate(fileName):
            if trainTestFlag:
                path = "/kaggle/working/train/" + p
                if os.path.exists(path):
                    if ind == 0:
                        fileNames.append((path, 0))
                    else:
                        fileNames.append((path, 1))
            else:
                path = "/kaggle/working/test/" + p
                if os.path.exists(path):
                    fileNames.append((path, -1))
    return fileNames

# Validation and Training Split

In [ ]:
fileNames = np.array(loadFileNames())
indeces = np.random.choice(len(fileNames), len(fileNames), replace=False)
trainTestRatio = 0.8
trainIndeces, validationIndeces = indeces[:int(0.8 * len(indeces))], indeces[int(0.8 * len(indeces)):] 
fileNamesTest = loadFileNames(False, 1, 12501)


# Data Augmentation and Data Loading

In [ ]:
class DataAugmentation(Dataset):

    def __init__(self, imgSize, fileNames, *args, **kwargs):
        super(DataAugmentation, self).__init__(*args, **kwargs)
        self.files = fileNames
        self.tensorTransform = torchvision.transforms.ToTensor()
        self.resize = torchvision.transforms.Resize((imgSize, imgSize))
        self.tranlationTransform = torchvision.transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))
        self.rotationalTransformation = torchvision.transforms.RandomRotation(45, center=(imgSize/2, imgSize/2) )
        self.imgSize = imgSize
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img = PIL.Image.open(self.files[index][0])
        label = self.files[index][1]
        img = self.resize(img)
        img = self.tranlationTransform(img)
        img = self.rotationalTransformation(img)
        #img = np.array(img, dtype=np.float32)
        img = self.tensorTransform(img)#Will read it into 3xheightxwidth
        #Standard normalize images
        ravel = img.view(3, -1)
        means = torch.mean(ravel, dim=1)
        ravel -= torch.transpose(means.repeat(ravel.shape[1]).reshape(-1, 3), 0, 1)
        std = torch.std(ravel, dim=1)
        ravel /= torch.transpose(std.repeat(ravel.shape[1]).reshape(-1, 3), 0, 1)
        
        img = ravel.view(3, self.imgSize, self.imgSize)

        return img, label

class OriginalImages(Dataset):

    def __init__(self, imgSize, fileNames, *args, **kwargs):
        super(OriginalImages).__init__(*args, **kwargs)
        self.files = fileNames
        self.tensorTransform = torchvision.transforms.ToTensor()
        self.resize = torchvision.transforms.Resize((imgSize, imgSize))
        self.imgSize = imgSize
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img = PIL.Image.open(self.files[index][0])
        label = self.files[index][1]
        img = self.resize(img)
        img = self.tensorTransform(img)
        
        #Standard normalize images
        ravel = img.view(3, -1)
        means = torch.mean(ravel, dim=1)
        ravel -= torch.transpose(means.repeat(ravel.shape[1]).reshape(-1, 3), 0, 1)
        std = torch.std(ravel, dim=1)
        ravel /= torch.transpose(std.repeat(ravel.shape[1]).reshape(-1, 3), 0, 1)
        
        img = ravel.view(3, self.imgSize, self.imgSize)
        return img, label

In [ ]:
augmentedDataTrain = DataAugmentation(224, fileNames[trainIndeces])
originalDataTrain = OriginalImages(224, fileNames[trainIndeces])
originalDataValidation = OriginalImages(224, fileNames[validationIndeces])
originalDataTest =  OriginalImages(224, fileNamesTest)

dataLoaderTrain = DataLoader(ConcatDataset([originalDataTrain, augmentedDataTrain]), batch_size=256, shuffle=True)
dataLoaderValid = DataLoader(originalDataValidation, batch_size=256, shuffle=True)
dataLoaderTest = DataLoader(originalDataTest, batch_size=256)

# Creating VGG13 from Scratch and Using Custom model with the pretrained model of the VGG16

In [ ]:
class VGG13(torch.nn.Module):

    def __init__(self, *args, **kwargs):
        super(VGG13, self).__init__(*args, **kwargs)
        self.conv1 = torch.nn.Conv2d(3, 64, (3, 3), 1)
        self.conv2 = torch.nn.Conv2d(64, 64, (3, 3), 1)
        self.pool1 = torch.nn.MaxPool2d((2, 2), 2)#non-overlapping
        self.conv3 = torch.nn.Conv2d(128, 128, (3, 3), 1)
        self.conv4 = torch.nn.Conv2d(128, 128, (3, 3), 1)
        self.pool2 = torch.nn.MaxPool2d((2, 2), 2)#non-overlapping
        self.conv5 = torch.nn.Conv2d(256, 256, (3, 3), 1)
        self.conv6 = torch.nn.Conv2d(256, 256, (3, 3), 1)
        self.pool3 = torch.nn.MaxPool2d((2, 2), 2)#non-overlapping
        self.conv7 = torch.nn.Conv2d(512, 512, (3, 3), 1)
        self.conv8 = torch.nn.Conv2d(512, 512, (3, 3), 1)
        self.pool4 = torch.nn.MaxPool2d((2, 2), 2)#non-overlapping
        self.conv9 = torch.nn.Conv2d(512, 512, (3, 3), 1)
        self.conv10 = torch.nn.Conv2d(512, 512, (3, 3), 1)
        self.pool5 = torch.nn.MaxPool2d((2, 2), 2)#non-overlapping

        self.fc1 = torch.nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = torch.nn.Linear(4096, 4096)
        self.fc3 = torch.nn.Linear(4096, 2)


    
    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.conv2(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.pool1(x)
        #print(x.shape)
        x = self.conv3(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.conv4(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.pool2(x)
        #print(x.shape)

        x = self.conv5(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.conv6(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.pool3(x)
        #print(x.shape)

        x = self.conv7(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.conv8(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.pool4(x)
        #print(x.shape)

        x = self.conv9(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.conv10(x)
        x = torch.relu(F.pad(x, (1, 1, 1, 1)))
        x = self.pool5(x)
        #print(x.shape)

        x = x.view(-1, 512 * 7 * 7)
        #print(x.shape)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        x = F.softmax(x, dim=2)

        return x
    
    @staticmethod
    def init_param(layer):
        if type(layer) == torch.nn.Conv2d:
            torch.nn.init.xavier_normal_(layer.weight)
        if type(layer) == torch.nn.Linear:
            torch.nn.init.xavier_uniform_(layer.weight)
    
    def initialize_parameters(self, verbose=False):
        self.apply(self.init_param)
        numberParameters = 0
        for p in self.parameters():
            numberParameters += p.numel() if p.requires_grad else 0
        if verbose:
            counter = 0
            for param in self.parameters():
                print(f"Layer {counter}")
                print(param)
                counter += 1
        print("Number of parameters is {:,}".format(numberParameters))

In [ ]:
class VggWithCustomLayers(torch.nn.Module):

    def __init__(self, *args, **kwargs):
        super(VggWithCustomLayers, self).__init__(*args, **kwargs)
        self.vgg = torchvision.models.vgg16(pretrained=True)
        for param in self.vgg.parameters():
            param.requires_grad = False
        in_features = self.vgg.classifier[-1].in_features
        block = torch.nn.Sequential(OrderedDict([
            ("conv_1", torch.nn.Linear(in_features, 2)),
            #("non-linear-1", torch.nn.ReLU()),
            #("last", torch.nn.Linear(128, 1)),
            ("softmax", torch.nn.Softmax(dim=1))
        ]))
        self.vgg.classifier[-1] = block

    def forward(self, x):
        x = self.vgg(x)
        return x

def predict(loader, model, y_pred, y_true, images):
    counter = 0
    for batch, labels in loader:
        gc.collect()
        y_p = model(batch)
        y = list(np.array(labels, dtype=np.int8))
        _, y_p_max = torch.max(y_p, dim=1)
        y_pred.extend(y_p_max.tolist())
        y_true.extend(y)
        print(f"batch#{counter}")
        if counter < 10:
            images.append(batch[0]) 
        del batch
        del y_p
        del y
        del y_p_max
        del _
        counter += 1

# Instantiation of the Model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

model = VggWithCustomLayers()
model.load_state_dict(torch.load("/kaggle/working/vgg_cat_vs_dog.h5"))
print(model)

#model.initialize_parameters(False)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()
validation_errors = []
training_errors = []
print(model)
model =model.to(device)
for name, param in model.named_parameters():
    print(f"{name} {param.requires_grad}")

# Training of the Model

In [ ]:
epochs = 1
def train_model(epochs, model, dataLoaderTrain, dataLoaderValid):
    y_pred_train = []
    y_pred_valid = []
    y_true_train = []
    y_true_valid = []
    for epoch in range(0, epochs):
        batchNum = 0
        begin = time.time()
        trainingErrorBatches = []
        validationErrorBatches = []

        for batch, labels in dataLoaderTrain:
            batch = batch.to(device)
            optimizer.zero_grad()
            y_pred = model(batch)
            y = torch.tensor(np.array(labels, dtype=np.int8)).type(torch.LongTensor)
            y = y.to(device)
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            if epoch + 1 == epochs:
                y_pred_train.extend(list(np.argmax(y_pred.detach().numpy(), axis=1)))
                y_true_train.extend(y.tolist())
            trainingErrorBatches.append(loss.item())
            print(f"{epoch}:{batchNum}, -log-likelihood {np.round(loss.item(), 3)}")
            cnf = confusion_matrix(y, np.argmax(y_pred.detach().numpy(), axis=1))
            print(f"Accuracy: {np.round(np.sum(np.diag(cnf))/np.sum(cnf), 3)*100}, \n {cnf}")
            batchNum += 1

        for batch, labels in dataLoaderValid:
            optimizer.zero_grad()
            y = torch.tensor(np.array(labels, dtype=np.int8)).type(torch.LongTensor)
            y_pred = model(batch)
            y = y.to(device)
            loss = loss_fn(y_pred, y)
            if epoch + 1 == epochs:
                y_pred_valid.extend(list(np.argmax(y_pred.detach().numpy(), axis=1)))
                y_true_valid.extend(y.tolist())
            validationErrorBatches.append(loss.item())
        training_errors.append(np.mean(trainingErrorBatches))
        validation_errors.append(np.mean(validationErrorBatches))

        end = time.time()
        print(f"{epoch} the avg -log-likelihood for training is {np.round(training_errors[-1], 2)} and it took {(end - begin)/60} min")
        print(f"{epoch} the avg -log-likelihood for validation is {np.round(validation_errors[-1], 2)} and it took {(end - begin)/60} min")
    return y_pred_train, y_true_train, y_pred_valid, y_true_valid

# Model Performance on the Training Set

In [ ]:
# Training Performance
y_pred = []
y_true = [] 
images = [] 
#predict(dataLoaderTrain, model, y_pred, y_true, images)
#cnf = confusion_matrix(y_pred, y_true)
#cnf = confusion_matrix(y_pred_train, y_true_train)
#print(f"Accuracy on training set {np.round(np.sum(np.diag(cnf))/np.sum(cnf), 3) * 100} \n {cnf}")

In [ ]:
# gs = GridSpec(2, 5)
# gs.update(left=0.12, bottom=0.08, right=2, top=0.92, wspace=0.2, hspace=0.5)
# lab = ["cat", "dog"]
# colors = ["red", "green"]
# for i in range(0, 2):
#     for j in range(0, 5):
#         ax = plt.subplot(gs[i, j])
#         ax.imshow(images[i*2 + j].transpose(0, 2)[:, :, 0], cmap="gray")
#         ax.set_xticks([])
#         ax.set_yticks([])
#         ax.set_title(f"y_true: {lab[y_true[i* 2 + j]]} and y_pred: {lab[y_pred[i*2 +j]]}", color=colors[(y_true[i* 2 + j] == y_pred[i*2 +j])*1] )
        
# plt.show()

# Model Performance on the Validation Set

In [ ]:
# del y_pred
# del y_true
# del images 
# del dataLoaderTrain
# del cnf

# Validation Performance
y_pred = []
y_true = [] 
images = [] 
predict(dataLoaderValid, model, y_pred, y_true, images)
#cnf = confusion_matrix(y_pred_valid, y_true_valid)
cnf = confusion_matrix(y_pred, y_true)
print(f"Accuracy on validation set {np.round(np.sum(np.diag(cnf))/np.sum(cnf), 3) * 100} \n {cnf}")

In [ ]:
gs = GridSpec(2, 5)
gs.update(left=0.12, bottom=0.08, right=2, top=0.92, wspace=0.2, hspace=0.5)
lab = ["cat", "dog"]
colors = ["red", "green"]
for i in range(0, 2):
    for j in range(0, 5):
        ax = plt.subplot(gs[i, j])
        ax.imshow(images[i*2 + j].transpose(0, 2)[:, :, 0], cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"y_true: {lab[y_true[i* 2 + j]]} and y_pred: {lab[y_pred[i*2 +j]]}", color=colors[(y_true[i* 2 + j] == y_pred[i*2 +j])*1] )
        
plt.show()

# Model Performance on the Test Set

In [ ]:
del dataLoaderValid
#del cnf
# Test Performance
y_pred = []
y_true = [] 
images = [] 
predict(dataLoaderTest, model, y_pred, y_true, images)
df = pd.DataFrame(np.c_[np.arange(1, len(y_pred) + 1), y_pred], columns=["id", "label"])
df.to_csv("submission.csv", header=True, index=False)

In [ ]:
gs = GridSpec(2, 5)
gs.update(left=0.12, bottom=0.08, right=2, top=0.92, wspace=0.2, hspace=0.5)
lab = ["cat", "dog"]
for i in range(0, 2):
    for j in range(0, 5):
        ax = plt.subplot(gs[i, j])
        ax.imshow(images[i*2 + j].transpose(0, 2)[:, :, 0], cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"y_pred: {lab[y_pred[i*2 +j]]}")
        
plt.show()

In [ ]:
# To download file, from https://www.kaggle.com/rtatman/download-a-csv-file-from-a-kernel
#from IPython.display import HTML
# import base64
#html = '<a  href="{filename}" target="_blank">{title}</a>'
#html = html.format(title="file",filename="submission.csv")
#HTML(html)

# References 
* https://arxiv.org/abs/1409.1556